In [1]:
import sys
project = "/home/weber/PycharmProjects/ExoCarto/"
sys.path.insert(0, project)
# IMPORTS
# import plotly.io as pio
# pio.renderers.default='notebook'
import plotly.express as px
import collections
import multiprocessing
import warnings
import plotly.graph_objects as go
import igv
import matplotlib.pyplot as plt
%matplotlib inline
import parmap 
import multiprocessing
import numpy as np
import ptitprince as pt
import seaborn as sns
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from src.utils import utils
import _pickle
import pandarallel
from pprint import pprint
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'from pprint import pprint
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
pandarallel.pandarallel.initialize(nb_workers=os.cpu_count(), progress_bar=True)

# CONFIG
config = utils.load_config_file(project + "src/config.yaml")

pathogenic_genes_file = project + config['ExtractDiseaseGenes']['DISEASE_GENES_PATH']
refseq = project + config['RefSeq']['df_transposed_gene_file']


INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [17]:
refseq = pd.read_csv(project + config['CCRS_analysis']['RefSeq_transformed'], sep='\t', compression='gzip')
refseq

,Gene,Chrom,Start,End,ranges,Length,GeneID,HGNC,CDS_representation,Ratio,Sharing_status,Sources,Count_CDS_alternative,Count_CDS_constitutive,mRNA_IDS,mRNA_nb
0,OR4F5,1,69091,70008,69091-70008,917,79501,14825.0,Unique,1.0/1,False,"[(69091, 70008, '1.0/1')]",0,1,['NM_001005484.1'],1
1,OR4F29,1,367659,368597,367659-368597,938,729759,31275.0,Unique,1.0/1,False,"[(367659, 368597, '1.0/1')]",0,1,['NM_001005221.2'],1
2,OR4F16,1,621096,622034,621096-622034,938,81399,15079.0,Unique,1.0/1,False,"[(621096, 622034, '1.0/1')]",0,1,['NM_001005277.1'],1
3,SAMD11,1,861322,861393,861322-861393,71,148398,28706.0,Unique,1.0/1,False,"[(861322, 861393, '1.0/1')]",0,13,['NM_152486.3'],1
4,SAMD11,1,865535,865716,865535-865716,181,148398,28706.0,Unique,1.0/1,False,"[(865535, 865716, '1.0/1')]",0,13,['NM_152486.3'],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238564,CDY1,Y,27768590,27770181,27768590-27770181,1591,9085,1809.0,Variable_region,2.0/2,True,"[(27768590, 27770181, '1.0/2'), (27768590, 277...",2,1,"['NM_004680.2', 'NM_170723.1']",2
238565,CDY1,Y,27770181,27770212,27770181-27770212,31,9085,1809.0,Variable_region,1.0/2,True,"[(27768590, 27770212, '1.0/2')]",2,1,"['NM_004680.2', 'NM_170723.1']",2
238566,CDY1,Y,27770181,27770212,27770181-27770212,31,9085,1809.0,Variable_region,1.0/2,True,"[(27768590, 27770212, '1.0/2')]",2,1,"['NM_004680.2', 'NM_170723.1']",2
238567,CDY1,Y,27770602,27770674,27770602-27770674,72,9085,1809.0,Unique,1.0/2,False,"[(27770602, 27770674, '1.0/2')]",2,1,"['NM_004680.2', 'NM_170723.1']",2


In [19]:
refseq.loc[refseq['Gene'] == "TP53", 'ranges']

83580    7572927-7573008
83581    7573927-7574033
83582    7576537-7576584
83583    7576625-7576657
83584    7576853-7576926
83585    7577019-7577155
83586    7577499-7577608
83587    7578177-7578289
83588    7578371-7578452
83589    7578452-7578533
83590    7578533-7578554
83591    7579312-7579569
83592    7579569-7579590
83593    7579700-7579721
83594    7579839-7579912
Name: ranges, dtype: object

In [3]:
ccrs = pd.read_csv(project + config['GeneExonLevel']['ccrs_file'], compression='gzip', sep='\t')
ccrs

,#chrom,start,end,ccr_pct,gene,ranges,varflag,syn_density,cpg,cov_score,resid,resid_pctile,unique_key
0,1,861263,861264,0.749507,AL645608.1,861263-861264,VARFALSE,0.0,0.0,0.98,-0.752,2.725378,3546161
1,1,861264,861265,0.000000,AL645608.1,861264-861265,VARTRUE,0.0,0.0,0.00,-2.075,0.000000,4099565
2,1,861265,861268,9.178962,AL645608.1,861265-861268,VARFALSE,0.0,0.0,2.94,-0.475,3.295490,2374016
3,1,861268,861269,0.000000,AL645608.1,861268-861269,VARTRUE,0.0,0.0,0.00,-2.075,0.000000,4099566
4,1,861269,861273,13.809869,AL645608.1,861269-861273,VARFALSE,0.0,0.0,3.92,-0.336,3.580546,2035627
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8188404,9,141016428,141016429,0.000000,CACNA1B,141016428-141016429,VARTRUE,0.0,0.0,0.00,-2.075,0.000000,7989550
8188405,9,141016429,141016440,51.824484,CACNA1B,141016429-141016440,VARFALSE,0.0,0.0,10.56,0.602,5.511947,631094
8188406,9,141016440,141016441,0.000000,CACNA1B,141016440-141016441,VARTRUE,0.0,0.0,0.00,-2.075,0.000000,7989551
8188407,9,141016441,141016448,47.211557,CACNA1B,"141016441-141016448,141016448-141016451","VARFALSE,VARFALSE",0.2,0.0,9.50,0.452,5.203621,738355


In [5]:
tp53 = ccrs.loc[ccrs['gene'] == 'TP53']
tp53.head(50)

,#chrom,start,end,ccr_pct,gene,ranges,varflag,syn_density,cpg,cov_score,resid,resid_pctile,unique_key
3336294,17,7572926,7572929,23.536853,TP53,"7572926-7572929,7572929-7572931","VARFALSE,VARFALSE",0.000,0.0,5.00,-0.184,3.894690,1537585
3336295,17,7572929,7572931,23.536853,TP53,"7572926-7572929,7572929-7572931","VARFALSE,VARFALSE",0.000,0.0,5.00,-0.184,3.894690,1537585
3336296,17,7572931,7572932,0.000000,TP53,7572931-7572932,VARTRUE,0.000,0.0,0.00,-2.075,0.000000,5627977
3336297,17,7572932,7572933,1.906723,TP53,7572932-7572933,VARFALSE,0.000,0.0,1.00,-0.749,2.731195,3261761
3336298,17,7572933,7572934,0.000000,TP53,7572933-7572934,VARTRUE,0.000,0.0,0.00,-2.075,0.000000,5627978
3336299,17,7572934,7572948,65.858213,TP53,7572934-7572948,VARFALSE,0.000,0.0,14.00,1.088,6.512552,359116
3336300,17,7572948,7572949,0.000000,TP53,7572948-7572949,VARTRUE,0.000,0.0,0.00,-2.075,0.000000,5627979
3336301,17,7572949,7572957,40.393402,TP53,7572949-7572957,VARFALSE,0.000,0.0,8.00,0.240,4.767310,922527
3336302,17,7572957,7572958,0.000000,TP53,7572957-7572958,VARTRUE,0.000,0.0,0.00,-2.075,0.000000,5627980
3336303,17,7572958,7572973,68.810227,TP53,7572958-7572973,VARFALSE,0.067,0.0,15.00,1.229,6.803425,310558
